In [ ]:
# !unzip Resume-markdown-docling-zip.zip

In [1]:
!pip install docling
!pip install -U sentence-transformers
!pip install rank-bm25
!pip install faiss-cpu
!pip install langchain
!pip install openai
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 72.4 MB/s eta 0:00:00
   ━

In [2]:
import os
import json
from pathlib import Path
from typing import Dict, List, Optional, Any
import hashlib
import statistics

from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from sentence_transformers import SentenceTransformer
import pickle
import re
from typing import List, Dict, Any
from rank_bm25 import BM25Okapi
from langchain_core.documents import Document

import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity


from google.colab import userdata
import openai

from openai import OpenAI



# Constants
RESUME_PDF_PATH = "Submit your resume or CV (File responses)"
RESUME_MARKDOWN_PATH = "Resume-markdown-docling"
MAX_RESUMES = 200

## PDF to Markdown Conversion using docling and saving it

In [ ]:
# # Basic PDF to Markdown conversion
# from pathlib import Path

# # Setup paths
# pdf_dir = Path("Submit your resume or CV (File responses)")
# output_dir = Path("Resume-markdown-docling")
# output_dir.mkdir(exist_ok=True)

# # Get first 200 PDFs
# pdf_files = list(pdf_dir.glob("*.pdf"))[:200]
# print(f"Found {len(pdf_files)} PDFs to convert")

# # Convert PDFs
# converter = DocumentConverter()
# successful = 0

# for pdf_file in pdf_files:
#     try:
#         result = converter.convert(str(pdf_file))
#         markdown_content = result.document.export_to_markdown()

#         output_file = output_dir / f"{pdf_file.stem}.md"
#         output_file.write_text(markdown_content, encoding='utf-8')

#         successful += 1
#         if successful % 20 == 0:
#             print(f"Converted {successful} files")

#     except Exception as e:
#         print(f"Failed: {pdf_file.name}")

# print(f"Conversion complete: {successful}/{len(pdf_files)} successful")

Found 52 PDFs to convert


[INFO] 2025-10-17 06:21:03,282 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-17 06:21:03,288 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-10-17 06:21:04,564 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2025-10-17 06:21:04,732 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-10-17 06:21:04,734 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-10-17 06:21:04,975 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-17 06:21:04,977 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/cls/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-10-17 06:21:06,332 [RapidOCR] downlo

## Chunking the markdowns based on custom rules
=> 3+ separated capital letters to be joined => due to conversion between pdf and markdown

=> splitting based on common headers in resume such as experience , summary etc using regex + it needs to be after ## => thats how docling does it


In [ ]:
# markdown_dir=Path("Resume-markdown-docling")
# def fix_spaced_caps(s: str) -> str:
#     pattern = re.compile(r'(?<!\w)(?:[A-Z]\s+){2,}[A-Z](?!\w)')  # 3+ capital letters separated by spaces
#     def _join(m):
#         return m.group(0).replace(' ', '')
#     return pattern.sub(_join, s)

# def container_chunking(content: str, resume_id: str) -> List[Document]:
#     # Remove image tags
#     content = re.sub(r'<!-- image -->', '', content)
#     #Removing spacing S K I L L -> SKILL
#     # Fix spaced out words like "e x p e r i e n c e" or "E X P E R I E N C E"
#     # content = re.sub(r'\b(\w)\s+(\w)\s+(\w)(\s+\w)*\b', lambda m: re.sub(r'\s+', '', m.group()), content)
#     content = fix_spaced_caps(content)

#     containers = [
#         r'about\s*me', r'summary', r'profile', r'experience', r'work\s+experience',
#         r'education', r'skill[s]?', r'project[s]?', r'achievement[s]?', r'award[s]?',
#         r'publication[s]?', r'competition[s]?', r'hackathon[s]?'
#     ]
#     container_alt = '|'.join(containers)

#     # Anchor to line start, any H1–H6, match only the heading line
#     pattern = rf'(?=^#{{1,6}}\s*(?:.*\b(?:{container_alt})\b).*$)'
#     chunks = re.split(pattern, content, flags=re.IGNORECASE | re.MULTILINE)

#     # Filter empty chunks and create documents
#     docs = []
#     for i, chunk in enumerate(chunks):
#         chunk = chunk.strip()
#         if chunk and len(chunk) > 50:
#             doc = Document(
#                 page_content=chunk,
#                 metadata={'resume_id': resume_id, 'chunk_id': i}
#             )
#             docs.append(doc)

#     return docs

# # Process all resumes
# all_chunks = []
# for md_file in markdown_dir.glob("*.md"):
#     content = md_file.read_text(encoding='utf-8')
#     resume_id = md_file.stem
#     chunks = container_chunking(content, resume_id)
#     all_chunks.extend(chunks)

# print(f"Created {len(all_chunks)} chunks from {len(list(markdown_dir.glob('*.md')))} resumes")
# print(f"Average chunks per resume: {len(all_chunks) / len(list(markdown_dir.glob('*.md'))):.1f}")

Created 250 chunks from 52 resumes
Average chunks per resume: 4.8


Embedding Models

Creating embeddings of each chunk in the document(resume) list and storing them in a pickle file (so don't have to keep running it again)
PS only works on colab due to version differences

In [5]:
# #CREATING OPEN AI EMBEDDINGS FOR ALL CHUNKS

# os.environ["OPENAI_API_KEY"] = (
#     userdata.get("OPENAI_API_KEY") or userdata.get("openai_api_key")
# )
# client=OpenAI()
# def get_openai_embeddings(texts: list[str], model: str = "text-embedding-3-small", batch_size: int = 100) -> np.ndarray:
#     embeddings: list[np.ndarray] = []
#     for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
#         batch = texts[i : i + batch_size]
#         resp = client.embeddings.create(model=model, input=batch)
#         embeddings.extend([np.array(item.embedding, dtype=np.float32) for item in resp.data])
#     return np.vstack(embeddings) if embeddings else np.zeros((0, 0), dtype=np.float32)

# # prepare chunks (prefer in-memory all_chunks, else load saved chunks)
# try:
#     chunks = all_chunks  # type: ignore[name-defined]
# except NameError:
#     if os.path.exists("resume_chunks.pkl"):
#         with open("resume_chunks.pkl", "rb") as f:
#             chunks = pickle.load(f)
#     else:
#         raise RuntimeError("No in-memory chunks and resume_chunks.pkl not found.")

# chunk_texts = [doc.page_content for doc in chunks]
# print(f"Generating OpenAI embeddings for {len(chunk_texts)} chunks...")

# # choose "text-embedding-3-small" or "text-embedding-3-large"
# embeddings = get_openai_embeddings(chunk_texts, model="text-embedding-3-small", batch_size=100)

# # ====================================================================================================================
# # SAVING THE EMBEDDINGS OF CHUNKS AND STORING IN PICKLE FILE
# with open("resume_chunks_openai.pkl", "wb") as f:
#     pickle.dump(chunks, f)

# with open("resume_embeddings_openai.pkl", "wb") as f:
#     pickle.dump(embeddings, f)

# print(f"Saved {len(chunks)} chunks and embeddings -> resume_chunks_openai.pkl, resume_embeddings_openai.pkl")
# print(f"Embeddings shape: {embeddings.shape}")

# # # ====================================================================================================================
# # # SACHECKING PURELY BASED ON COSINE SIMILARITY

# # # retrieval function using cosine similarity
# # def retrieve_similar_chunks_openai(query: str, top_k: int = 5, model: str = "text-embedding-3-large"):
# #     # ensure embeddings & chunks are loaded in scope
# #     global embeddings, chunks
# #     if 'embeddings' not in globals() or embeddings is None or embeddings.size == 0:
# #         if os.path.exists("resume_embeddings_openai.pkl"):
# #             with open("resume_embeddings_openai.pkl", "rb") as f:
# #                 embeddings = pickle.load(f)
# #         else:
# #             raise RuntimeError("Embeddings not loaded. Run embedding generation first.")
# #     if 'chunks' not in globals() or chunks is None:
# #         if os.path.exists("resume_chunks_openai.pkl"):
# #             with open("resume_chunks_openai.pkl", "rb") as f:
# #                 chunks = pickle.load(f)
# #         else:
# #             raise RuntimeError("Chunks not loaded. Run embedding generation first.")


# #     0# get query embedding
# #     q_resp = client.embeddings.create(model=model, input=[query])
# #     q_emb = np.array(q_resp.data[0].embedding, dtype=np.float32).reshape(1, -1)
# #     sims = cosine_similarity(q_emb, embeddings)[0]
# #     top_idx = np.argsort(sims)[-top_k:][::-1]
# #     results = []
# #     for i in top_idx:
# #         results.append({
# #             "score": float(sims[i]),
# #             "content": chunks[i].page_content,
# #             "metadata": chunks[i].metadata
# #         })
# #     return results



In [ ]:
# # example usage:
# results = retrieve_similar_chunks_openai("student who has scored more than 95% in school", top_k=3)
# # for r in results: print(r["score"], r["metadata"]["resume_id"], r["content"][:200])

Trying Hybrid Retrieval


For resume based querying, just semantic search doesn't cut it, a lot of the times, the recruiters are looking for some keywords, with semantic similarity it can't match exact keywords hence we need to combine approaches; keyword matching +semantic similarity

In [3]:
 # Load saved chunks and embeddings
with open('resume_chunks_openai.pkl', 'rb') as f:
    chunks = pickle.load(f)

with open('resume_embeddings_openai.pkl', 'rb') as f:
    embeddings = pickle.load(f)

# --- tiny cleaner & tokenizer ---
_CLEAN_RX = re.compile(r'[^\w\s]')
_WS_RX = re.compile(r'\s+')

def clean_and_tokenize(text: str) -> List[str]:
    text = text.lower()
    text = _CLEAN_RX.sub(' ', text)     # remove punctuation
    text = _WS_RX.sub(' ', text).strip()
    return [t for t in text.split(' ') if t]

class BM25Index:
    def __init__(self):
        self.bm25 = None
        self.docs: List[Document] = []
        self.doc_tokens: List[List[str]] = []

    def fit(self, chunks: List[Document]):
        self.docs = chunks
        self.doc_tokens = [clean_and_tokenize(d.page_content) for d in chunks]
        self.bm25 = BM25Okapi(self.doc_tokens)

    def search(self, query: str, top_k: int = 10) -> List[Dict[str, Any]]:
        if self.bm25 is None:
            raise RuntimeError("Call fit(chunks) before search().")
        q_tokens = clean_and_tokenize(query)
        scores = self.bm25.get_scores(q_tokens)
        # top-k indices by score
        top_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        out = []
        for rank, i in enumerate(top_idx, 1):
            d = self.docs[i]
            out.append({
                "rank": rank,
                "bm25_score": float(scores[i]),
                "resume_id": (d.metadata or {}).get("resume_id"),
                "chunk_id": (d.metadata or {}).get("chunk_id"),
                "preview": d.page_content[:400]
            })
        return out


In [4]:
bm25 = BM25Index()
bm25.fit(chunks)  # chunks = your List[Document] from the chunker

query = "Urban Water Logging Detection for Timely Intervention and Mitigation"
hits = bm25.search(query, top_k=10)

print(f"BM25 hits: {len(hits)}")
for h in hits[:5]:
    print(f"[{h['rank']:>2}] {h['bm25_score']:.3f}  resume={h['resume_id']}  chunk={h['chunk_id']}")
    print("   ", (h["preview"] or "").splitlines()[0][:540], "...")


BM25 hits: 10
[ 1] 30.951  resume=Resume Mollika - Mollika Garg  chunk=4
    ## SELECTED PROJECTS ...
[ 2] 16.028  resume=Resume Mollika - Mollika Garg  chunk=5
    ## ACHIEVEMENTS ...
[ 3] 10.170  resume=Resume.PunithHM - Punith H M  chunk=2
    ## Projects ...
[ 4] 9.759  resume=Zulfikar_resume - Zulfikar Charoliya  chunk=2
    ## PROJECTS ...
[ 5] 5.784  resume=Aziz Bayu Pratama - Curriculum Vitae - 4IA04_Aziz Bayu Pratama  chunk=2
    ## EXPERIENCES ...


In [5]:

os.environ["OPENAI_API_KEY"] = (
    userdata.get("OPENAI_API_KEY") or userdata.get("openai_api_key")
)
client=OpenAI()

class DenseIndexSimple:
    def __init__(self):
        self.docs: List[Document] = []
        self.Xn: np.ndarray | None = None  # L2-normalized embeddings (N, D)
        self.dim: int | None = None
        self.model_name: str | None = None

    def fit(self, chunks: List[Document], embeddings: np.ndarray, model_name: str):
        """
        chunks: your List[Document]
        embeddings: np.ndarray of shape (N, D) aligned with chunks
        model_name: the embedding model used (e.g., 'text-embedding-3-small')
        """
        if embeddings.ndim != 2 or len(chunks) != embeddings.shape[0]:
            raise ValueError("Embeddings must be 2D and aligned with chunks.")
        X = embeddings.astype(np.float32)
        X = np.ascontiguousarray(X)
        norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
        self.Xn = X / norms
        self.dim = self.Xn.shape[1]
        self.docs = chunks
        self.model_name = model_name

    def _embed_query(self, query: str) -> np.ndarray:
        if not self.model_name:
            raise RuntimeError("Index not initialized with a model_name. Call fit() first.")
        resp = client.embeddings.create(model=self.model_name, input=[query])
        q = np.array(resp.data[0].embedding, dtype=np.float32)
        if q.shape[0] != self.dim:
            raise ValueError(f"Query dim {q.shape[0]} != index dim {self.dim}. "
                             f"Use the same embedding model as indexing.")
        q = q / (np.linalg.norm(q) + 1e-12)
        return q  # (D,)

    def search(self, query: str, top_k: int = 200) -> list[dict]:
        if self.Xn is None:
            raise RuntimeError("Index empty. Call fit() first.")

        # embed + normalize query
        q = self._embed_query(query)                 # (D,)
        sims = self.Xn @ q                           # (N,)
        n = sims.shape[0]
        k = max(1, min(top_k, n))                    # clamp to [1, N]

        # For small N, argsort is fine and simpler
        top_idx = np.argsort(sims)[::-1][:k]

        results = []
        for rank, i in enumerate(top_idx, 1):
            d = self.docs[i]
            results.append({
                "rank": rank,
                "dense_score": float(sims[i]),
                "resume_id": (d.metadata or {}).get("resume_id"),
                "chunk_id": (d.metadata or {}).get("chunk_id"),
                "preview": d.page_content[:400],
            })
        return results



In [6]:
# You already have:
# - chunks: List[Document]
# - embeddings: np.ndarray from your earlier OpenAI embedding step
# - model_name: e.g., "text-embedding-3-small" (must match the one used for embeddings)

dense = DenseIndexSimple()
dense.fit(chunks, embeddings, model_name="text-embedding-3-small")

query = "mumbai university"
dense_hits = dense.search(query, top_k=10)

print(f"Dense hits: {len(dense_hits)}")
for h in dense_hits[:5]:
    print(f"[{h['rank']:>2}] {h['dense_score']:.4f} resume={h['resume_id']} chunk={h['chunk_id']}")


Dense hits: 10
[ 1] 0.4334 resume=VES_Ganesh_Deulkar_Resume - Ganesh Deulkar chunk=1
[ 2] 0.3864 resume=SHAHNAWAZ_RESUME (4) - shahnawaz Shaikh chunk=2
[ 3] 0.3802 resume=Saaquib Motiwala Resume-6 (1) - saaquib motiwala chunk=3
[ 4] 0.3674 resume=Mohit_CV - Mohit Lohani chunk=1
[ 5] 0.3660 resume=Resume Mollika - Mollika Garg chunk=1


In [7]:
import hashlib
from typing import List, Dict, Any, Optional

def _make_key(hit: Dict[str, Any]) -> str:
    """Stable key to identify a chunk across lists."""
    rid = (hit.get("resume_id") or "").strip()
    cid = str(hit.get("chunk_id") or "").strip()
    if rid or cid:
        return f"{rid}::{cid}"
    # Fallback: hash preview if metadata missing
    prev = (hit.get("preview") or "")[:256]
    return "hash::" + hashlib.md5(prev.encode("utf-8")).hexdigest()

def rrf_fuse(
    bm25_hits: Optional[List[Dict[str, Any]]],
    dense_hits: Optional[List[Dict[str, Any]]],
    k: int = 60,
    top_k: int = 5,
    weights: Dict[str, float] = None,
) -> List[Dict[str, Any]]:
    """
    Reciprocal Rank Fusion (RRF): score = sum_s w_s * 1/(k + rank_s)
    - bm25_hits / dense_hits: lists with at least {'rank', 'resume_id', 'chunk_id'} (or 'preview').
    - k: stabilization constant (common choices: 60, 100).
    - top_k: number of fused results to return.
    - weights: optional per-source weights, e.g., {'bm25': 1.0, 'dense': 1.0}
    """
    weights = weights or {"bm25": 1.0, "dense": 1.0}
    pool: Dict[str, Dict[str, Any]] = {}

    def add_source(hits: Optional[List[Dict[str, Any]]], label: str):
        if not hits:
            return
        for h in hits:
            key = _make_key(h)
            rec = pool.setdefault(key, {
                "resume_id": h.get("resume_id"),
                "chunk_id": h.get("chunk_id"),
                "preview": h.get("preview"),
                # keep original per-source info if present
                "bm25_rank": None, "bm25_score": None,
                "dense_rank": None, "dense_score": None,
                "rrf_score": 0.0,
            })
            r = h.get("rank")
            if isinstance(r, int) and r >= 1:
                rec["rrf_score"] += weights.get(label, 1.0) * (1.0 / (k + r))
            # stash per-source details (first occurrence wins)
            rank_key = f"{label}_rank"
            score_key = f"{label}_score"
            if rec[rank_key] is None:
                rec[rank_key] = r
            if rec[score_key] is None:
                # hit may have 'bm25_score' or 'dense_score'
                val = h.get(score_key) or h.get("bm25_score") or h.get("dense_score")
                rec[score_key] = float(val) if val is not None else None

    add_source(bm25_hits, "bm25")
    add_source(dense_hits, "dense")

    fused = sorted(pool.values(), key=lambda x: x["rrf_score"], reverse=True)[:top_k]
    # add final rank
    for i, rec in enumerate(fused, 1):
        rec["rank"] = i
    return fused


In [8]:
query="Urban Water Logging Detection for Timely Intervention and Mitigation"
bm25_hits = bm25.search(query, top_k=10)
dense_hits = dense.search(query, top_k=10)

fused = rrf_fuse(bm25_hits, dense_hits, k=60, top_k=5, weights={"bm25": 2.0, "dense": 1.0})
for r in fused[:10]:
    print(f"[{r['rank']:>2}] RRF={r['rrf_score']:.5f}  bm25_r={r['bm25_rank']}  dense_r={r['dense_rank']}")
    print("   ", (r["preview"] or "").splitlines()[0][:120], "...")


[ 1] RRF=0.04918  bm25_r=1  dense_r=1
    ## SELECTED PROJECTS ...
[ 2] RRF=0.04788  bm25_r=3  dense_r=2
    ## Projects ...
[ 3] RRF=0.04528  bm25_r=8  dense_r=3
    ## (Google Earth Engine, EDA, LSTM, Project Management)                                                                  ...
[ 4] RRF=0.04459  bm25_r=6  dense_r=10
    ## PERSONAL PROJECTS ...
[ 5] RRF=0.04456  bm25_r=7  dense_r=8
    ## Professional Experience ...


In [9]:
for r in fused[:5]:
    print(r.get("resume_id", ""))

Resume Mollika - Mollika Garg
Resume.PunithHM - Punith H M
Md_Aamir - Md Aamir
Basit_s AI Resume - Basit Ali
Resume_Soham_Y - Soham Yedgaonkar


In [10]:
# import torch
# gpu_memory_bytes=torch.cuda.get_device_properties(0).total_memory
# gpu_memory_gb=round(gpu_memory_bytes/(2**30))
# print(gpu_memory_gb) #15 gb available to us!!!

In [25]:
from openai import OpenAI
from typing import List, Dict, Any

os.environ["OPENAI_API_KEY"] = (
    userdata.get("OPENAI_API_KEY") or userdata.get("openai_api_key")
)
client=OpenAI()

with open('resume_chunks_openai.pkl', 'rb') as f:
    all_chunks = pickle.load(f)

def gather_full_resumes(fused_results: List[Dict[str, Any]], all_chunks: List[Document]) -> Dict[str, str]:
    """Collect all chunks for each resume_id mentioned in fused results."""
    resume_ids = {r["resume_id"] for r in fused_results if r.get("resume_id")}

    resume_content = {}
    for rid in resume_ids:
        # Get all chunks for this resume
        resume_chunks = [c.page_content for c in all_chunks if c.metadata.get("resume_id") == rid]
        resume_content[rid] = "\n\n".join(resume_chunks)

    return resume_content

def summarize_resumes_with_llm(query: str, fused_results: List[Dict[str, Any]], all_chunks: List[Document], top_n: int = 5):
    """Generate summaries for top N resumes based on query."""

    # Get top N unique resume IDs
    top_resume_ids = []
    for r in fused_results:
        rid = r.get("resume_id")
        if rid and rid not in top_resume_ids:
            top_resume_ids.append(rid)
        if len(top_resume_ids) >= top_n:
            break

    # Gather full resume content
    resume_content = gather_full_resumes(fused_results, all_chunks)

    summaries = []
    for rid in top_resume_ids:
        full_resume = resume_content.get(rid, "")

        prompt = f"""You are a recruiter assistant. Analyze this resume and provide:
1. A brief summary of the candidate's profile (2-3 sentences)
2. How this candidate matches the query: "{query}"
3. Key strengths relevant to the query

Resume ID: {rid}

Resume:
{full_resume[:4000]}  # Limit to avoid token limits

Provide a concise response."""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=300
        )

        summary = response.choices[0].message.content
        summaries.append({
            "resume_id": rid,
            "summary": summary,
            "rrf_score": next((r["rrf_score"] for r in fused_results if r.get("resume_id") == rid), None)
        })

    return summaries


In [26]:

# Usage
query = "A candidate with background in deep learning and machine learning and have a cgpa above 9.5"
summaries = summarize_resumes_with_llm(query, fused, all_chunks, top_n=5)

for i, s in enumerate(summaries, 1):
    print(f"\n{'='*60}")
    print(f"RANK {i} | Resume ID: {s['resume_id']} | RRF Score: {s['rrf_score']:.5f}")
    print(f"{'='*60}")
    print(s['summary'])


RANK 1 | Resume ID: Atharva Domale Resume - Atharva Domale | RRF Score: 0.03126
1. The candidate is a Bachelor of Technology in Artificial Intelligence and Data Science with a GPA of 7/10. They have experience in developing and implementing machine learning models, data analysis projects, and certifications in AI and cloud platforms.

2. This candidate does not meet the query of having a CGPA above 9.5, as their GPA is 7/10.

3. Key strengths relevant to the query include a strong understanding of deep learning and machine learning technologies, proficiency in data analysis and model optimization, and certifications in AI and cloud platforms.

RANK 2 | Resume ID: Resume - Nahid Kawsar | RRF Score: 0.03110
1. Nahid Kawsar is an aspiring machine learning engineer with a strong foundation in applied mathematics, specializing in NLP, deep learning, and generative AI. They have experience in leveraging machine learning frameworks and data science tools to solve complex problems.

2. Nahid 

In [27]:
with open('resume_chunks_openai.pkl', 'rb') as f:
    all_chunks = pickle.load(f)

In [28]:
from sentence_transformers import CrossEncoder
import torch

# Load cross-encoder reranker
reranker = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

def rerank_with_cross_encoder(query: str, fused_results: List[Dict[str, Any]], all_chunks: List[Document], top_k: int = 10) -> List[Dict[str, Any]]:
    """Rerank fused results using cross-encoder for better relevance"""

    # Get full chunk content for each result
    pairs = []
    valid_results = []

    for r in fused_results:
        resume_id = r.get("resume_id")
        chunk_id = r.get("chunk_id")

        # Find matching chunk
        matching_chunk = next(
            (c for c in all_chunks if c.metadata.get("resume_id") == resume_id and c.metadata.get("chunk_id") == chunk_id),
            None
        )

        if matching_chunk:
            pairs.append((query, matching_chunk.page_content))
            valid_results.append(r)

    # Get cross-encoder scores
    ce_scores = reranker.predict(pairs).tolist()

    # Add cross-encoder scores to results
    for result, score in zip(valid_results, ce_scores):
        result["ce_score"] = float(score)

    # Sort by cross-encoder score
    reranked = sorted(valid_results, key=lambda x: x["ce_score"], reverse=True)[:top_k]

    # Update ranks
    for i, r in enumerate(reranked, 1):
        r["rerank_position"] = i

    return reranked



In [29]:
# Usage
query = "A candidate with a background in machine learning and deep learning and has a GPA above 8"
bm25_hits = bm25.search(query, top_k=10)
dense_hits = dense.search(query, top_k=10)
fused = rrf_fuse(bm25_hits, dense_hits, k=60, top_k=3)

# Rerank top 50 results to get final top 10
reranked = rerank_with_cross_encoder(query, fused, all_chunks, top_k=10)

print("RERANKED RESULTS:")
for r in reranked:
    print(f"[{r['rerank_position']:>2}] CE={r['ce_score']:.4f} RRF={r['rrf_score']:.5f} resume={r['resume_id']}")

RERANKED RESULTS:
[ 1] CE=-0.8739 RRF=0.03110 resume=Resume - Nahid Kawsar
[ 2] CE=-2.1342 RRF=0.03126 resume=Atharva Domale Resume - Atharva Domale
[ 3] CE=-6.8400 RRF=0.02899 resume=ATS_Resume_Dev - Dev Bhanushali


In [30]:
# Usage
query = "A candidate with background in deep learning and machine learning and have a cgpa above 9.5"
summaries = summarize_resumes_with_llm(query, reranked, all_chunks, top_n=5)

for i, s in enumerate(summaries, 1):
    print(f"\n{'='*60}")
    print(f"RANK {i} | Resume ID: {s['resume_id']} | RRF Score: {s['rrf_score']:.5f}")
    print(f"{'='*60}")
    print(s['summary'])


RANK 1 | Resume ID: Resume - Nahid Kawsar | RRF Score: 0.03110
1. The candidate, Nahid Kawsar, is an aspiring machine learning engineer with a strong foundation in applied mathematics, specializing in NLP, deep learning, and generative AI. They have experience in leveraging machine learning frameworks and data science tools to solve complex problems.

2. Nahid Kawsar matches the query for a candidate with a background in deep learning and machine learning as they have experience in deep learning with various neural network architectures, transfer learning, and generative AI. However, their current CGPA of 3.63 does not meet the requirement of above 9.5.

3. Key strengths relevant to the query include expertise in deep learning with various architectures such as ANN, CNN, RNN, LSTM, Transformers, and Attention Mechanism. They also have experience with transfer learning using models like VGG16, VGG19, AlexNet, ResNet, EfficientNet, and MobileNet. Additionally, their specialization in NL

In [31]:
import re
from typing import Optional, Dict, Any
from langchain_core.documents import Document

def extract_gpa(text: str) -> Optional[Dict[str, Any]]:
    """
    Extract GPA/CGPA from text with multiple pattern variations.
    Returns dict with gpa_value and gpa_scale, or None if not found.
    """
    # Common GPA patterns
    patterns = [
        # CGPA: 9.2/10, GPA: 3.8/4.0, CGPA 8.5 / 10
        r'(?:cgpa|gpa|cpi|spi)\s*[:\-]?\s*(\d+\.?\d*)\s*(?:/|out of)\s*(\d+\.?\d*)',

        # 9.2 CGPA, 3.8 GPA
        r'(\d+\.?\d*)\s+(?:cgpa|gpa|cpi|spi)',

        # CGPA 9.2, GPA 3.8
        r'(?:cgpa|gpa|cpi|spi)\s+(\d+\.?\d*)',

        # Percentage: 85%, 92.5%
        r'(\d+\.?\d*)\s*%',

        # Grade: 9.2 (common in bullet points)
        r'(?:grade|score|marks?)\s*[:\-]?\s*(\d+\.?\d*)\s*(?:/|out of)\s*(\d+\.?\d*)',
    ]

    results = []

    for pattern in patterns:
        matches = re.finditer(pattern, text, re.IGNORECASE)
        for match in matches:
            if len(match.groups()) == 2:
                # Has scale: 9.2/10
                gpa_value = float(match.group(1))
                gpa_scale = float(match.group(2))
            else:
                # No scale: just "9.2 CGPA"
                gpa_value = float(match.group(1))
                # Infer scale based on value
                if gpa_value <= 4.5:
                    gpa_scale = 4.0
                elif gpa_value <= 10.5:
                    gpa_scale = 10.0
                elif gpa_value <= 100:
                    gpa_scale = 100.0  # Percentage
                else:
                    continue  # Invalid value

            # Validate ranges
            if 0 <= gpa_value <= gpa_scale:
                results.append({
                    "gpa_value": gpa_value,
                    "gpa_scale": gpa_scale,
                    "gpa_normalized": round(gpa_value / gpa_scale * 10, 2),  # Normalize to /10
                    "match_text": match.group(0)
                })

    # Return first valid match
    return results[0] if results else None

def add_gpa_to_chunks(chunks: List[Document]) -> List[Document]:
    """Add GPA metadata to each chunk"""
    enriched_chunks = []

    for chunk in chunks:
        gpa_info = extract_gpa(chunk.page_content)

        # Create new metadata dict with GPA info
        new_metadata = chunk.metadata.copy()
        if gpa_info:
            new_metadata["gpa_value"] = gpa_info["gpa_value"]
            new_metadata["gpa_scale"] = gpa_info["gpa_scale"]
            new_metadata["gpa_normalized"] = gpa_info["gpa_normalized"]
            new_metadata["gpa_found"] = True
        else:
            new_metadata["gpa_value"] = None
            new_metadata["gpa_scale"] = None
            new_metadata["gpa_normalized"] = None
            new_metadata["gpa_found"] = False

        # Create new Document with updated metadata
        enriched_chunk = Document(
            page_content=chunk.page_content,
            metadata=new_metadata
        )
        enriched_chunks.append(enriched_chunk)

    return enriched_chunks

def analyze_gpa_coverage(chunks: List[Document]) -> Dict[str, Any]:
    """Analyze GPA extraction coverage across resumes"""

    # Group by resume_id
    resumes_with_gpa = set()
    resumes_without_gpa = set()
    total_chunks_with_gpa = 0
    gpa_values = []

    for chunk in chunks:
        resume_id = chunk.metadata.get("resume_id")

        if chunk.metadata.get("gpa_found"):
            resumes_with_gpa.add(resume_id)
            total_chunks_with_gpa += 1
            gpa_values.append(chunk.metadata.get("gpa_normalized"))
        else:
            if resume_id not in resumes_with_gpa:
                resumes_without_gpa.add(resume_id)

    # Remove resumes from "without" set if they have GPA in any chunk
    resumes_without_gpa -= resumes_with_gpa

    total_resumes = len(resumes_with_gpa) + len(resumes_without_gpa)
    coverage_pct = (len(resumes_with_gpa) / total_resumes * 100) if total_resumes > 0 else 0

    return {
        "total_resumes": total_resumes,
        "resumes_with_gpa": len(resumes_with_gpa),
        "resumes_without_gpa": len(resumes_without_gpa),
        "coverage_percentage": round(coverage_pct, 2),
        "total_chunks_with_gpa": total_chunks_with_gpa,
        "avg_gpa_normalized": round(sum(gpa_values) / len(gpa_values), 2) if gpa_values else None,
        "min_gpa": min(gpa_values) if gpa_values else None,
        "max_gpa": max(gpa_values) if gpa_values else None,
    }

# ============================================================
# USAGE
# ============================================================

# Load your chunks
with open('resume_chunks_openai.pkl', 'rb') as f:
    chunks = pickle.load(f)

print(f"Original chunks: {len(chunks)}")

# Add GPA metadata
enriched_chunks = add_gpa_to_chunks(chunks)

# Analyze coverage
coverage = analyze_gpa_coverage(enriched_chunks)

print("\n" + "="*60)
print("GPA EXTRACTION COVERAGE REPORT")
print("="*60)
print(f"Total Resumes: {coverage['total_resumes']}")
print(f"Resumes with GPA: {coverage['resumes_with_gpa']}")
print(f"Resumes without GPA: {coverage['resumes_without_gpa']}")
print(f"Coverage: {coverage['coverage_percentage']}%")
print(f"\nTotal chunks with GPA: {coverage['total_chunks_with_gpa']}")
print(f"Average GPA (normalized /10): {coverage['avg_gpa_normalized']}")
print(f"GPA Range: {coverage['min_gpa']} - {coverage['max_gpa']}")

# Show some examples
print("\n" + "="*60)
print("SAMPLE EXTRACTIONS (First 5 with GPA)")
print("="*60)
count = 0
for chunk in enriched_chunks:
    if chunk.metadata.get("gpa_found") and count < 5:
        print(f"\nResume: {chunk.metadata['resume_id']}")
        print(f"GPA: {chunk.metadata['gpa_value']}/{chunk.metadata['gpa_scale']} (Normalized: {chunk.metadata['gpa_normalized']}/10)")
        print(f"Preview: {chunk.page_content[:200]}...")
        count += 1

# Save enriched chunks
with open('resume_chunks_with_gpa.pkl', 'wb') as f:
    pickle.dump(enriched_chunks, f)

print(f"\n✅ Saved enriched chunks to 'resume_chunks_with_gpa.pkl'")

Original chunks: 249

GPA EXTRACTION COVERAGE REPORT
Total Resumes: 52
Resumes with GPA: 33
Resumes without GPA: 19
Coverage: 63.46%

Total chunks with GPA: 54
Average GPA (normalized /10): 7.43
GPA Range: 0.18 - 9.99

SAMPLE EXTRACTIONS (First 5 with GPA)

Resume: zaid_resume_data - Zaid Ahmed Khan
GPA: 95.6/100.0 (Normalized: 9.56/10)
Preview: ## EDUCATION

|   Year | Degree/Exam                 | Institute                                  | CGPA/Marks   |
|--------|-----------------------------|--------------------------------------------|...

Resume: zaid_resume_data - Zaid Ahmed Khan
GPA: 97.0/100.0 (Normalized: 9.7/10)
Preview: ## COMPETITION/CONFERENCE

## Automatic liquid detecting system | Computer Vision | ML | Empower IIT Madras| Prof. P K Dan (Gold) Link



- Innovated a Computer-Vision based product for liquid volume ...

Resume: zaid_resume_data - Zaid Ahmed Khan
GPA: 56.2/100.0 (Normalized: 5.62/10)
Preview: ## PROJECTS

## Hangman Challenge | Reinforcement Learning | Na

In [32]:
with open('resume_chunks_with_gpa.pkl', 'rb') as f:
    gpa_chunks = pickle.load(f)

In [33]:
gpa_chunks

[Document(metadata={'resume_id': 'zaid_resume_data - Zaid Ahmed Khan', 'chunk_id': 0, 'gpa_value': None, 'gpa_scale': None, 'gpa_normalized': None, 'gpa_found': False}, page_content='## ZAID AHMED KHAN | https://www.linkedin.com/in/zaid-ahmed-khan-/\n\nPhone No. +918770642550 | Email Idik241168@gmail.com\n\nChemical Engineering (B.Tech+M.Tech),Artificial Intelligence (Micro)'),
 Document(metadata={'resume_id': 'zaid_resume_data - Zaid Ahmed Khan', 'chunk_id': 1, 'gpa_value': 95.6, 'gpa_scale': 100.0, 'gpa_normalized': 9.56, 'gpa_found': True}, page_content='## EDUCATION\n\n|   Year | Degree/Exam                 | Institute                                  | CGPA/Marks   |\n|--------|-----------------------------|--------------------------------------------|--------------|\n|   2026 | Dual Degree (B.Tech+M.Tech) | IIT Kharagpur                              | 8.80/ 10     |\n|   2021 | CLASS XII                   | Christ Church Boys Senior Secondary School | 95.60%       |\n|   2019 | C